In [ ]:
import os
import pandas as pd
import geopandas as gp
import matplotlib.pyplot as plt

# UPDATE 9.3.2020: Revised data from NYWSC. Now making well file both for current SS rates and max capacity

Here is the readme from the OneDrive folder:

## ReadMe: WATERUSE_RONDOUT_NEVERSINK_2010_2015_Avg     5/4/2020

This shapefile includes all wells in our study area that have a pumpage associated.

### Fields:

__ID_Well__ = WWR# and Well name joined together

__Type__ = Type of well (all groundwater)

__Facility__ = WWR# (obtained this from SWUDS)

__Site_Name__ = Well name

__X__ = Longitude (NAD83)

__Y__ = Latitude (NAD83)

__Location_C__ = Location of well, or in situations where well location is unknown, defaulted to factility location.

__Avg_Mdg__ = 2010-2015 average pumpaging per well, in MGD. Pumping rates were obtained by distributing total facility pumpage amongst all wells within facility (Percentage determined by well capacities)

__Well_Depth__ = Well depth, in feet

__NWIS_Link__ = Well number in NWIS (these may not be 100% accurate)

__NWIS_ID__ = Site ID in NWIS (these may not be 100% accurate)

__WellDMeter__ = Well depth, in meters

__Stop_Meter__ = Screen top elevation, in meters

__Sbot_Meter__ = Screen bottom elevation, in meters

__Comments__ = Any comments pertinent to the well

__RASTERVALU__ = Lidar elevation of well, in meters

__WellType__ = Priority (as identified by DEC), Not priority (as identified by DEC), and Other (additional pumping wells from SWUDS)


### READ THIS


It is important to note that all rates, locations, and screen tops and bottoms are subject to change.
Only locations for priority wells have been checked for accuracy. 
Screen tops and bottoms were all unknown.  Well screen defaulted to bottom of well, as a 10 ft screen. We may have slightly adjusted this to fit the screen entirely in a specific layer.
Well shapefile currently contains both bedrock wells and non-bedrock wells. I'm not sure if we are excluding bedrock wells from the shapefile. 

In [ ]:
#wu_file = 'WATERUSE_RONDOUT_NEVERSINK_2010_2015_Avg.shp'
#wu_dir = '../sciencebase/Shapefiles/WaterUse_2010_2015Average/'
wu_file = 'WaterUse_With_Max_Capacities.shp'
wu_dir = '../sciencebase/Shapefiles/WaterUse/'

wu_path = os.path.join(wu_dir, wu_file)

boundary = gp.read_file('../sciencebase/Shapefiles/Extents/Model_Extent_HUC12.shp')

In [ ]:
gdf = gp.read_file(wu_path)

In [ ]:
fig, ax = plt.subplots()
gdf.plot(ax=ax, markersize=10)
boundary.plot(ax=ax, facecolor='none', edgecolor='red')
plt.show()

In [ ]:
gdf.columns

###  things to work up for wel file
* add columns with projected x-y from geometry column
* add top of screen elevation above model
* add bottom of screen elevation = 0 m
* start and end are arb

want columns: node, screen_top, screen_botm, flux_m3, start_datetime, end_datetime, x, y

In [ ]:
well_df = gdf.copy()

In [ ]:
well_df['x'] = [coord.x for coord in gdf.geometry]
well_df['y'] = [coord.y for coord in gdf.geometry]


In [ ]:
well_df.loc[(well_df.Stop_Meter == 0) & (well_df.Sbot_Meter == 0), 'Stop_Meter' ] = 2000
well_df.loc[(well_df.Stop_Meter == 2000) & (well_df.Sbot_Meter == 0), 'Sbot_Meter' ] = -2000


In [ ]:
well_df

##### confirming units because Mars rovers
$\frac{MG}{d} --> \frac{m^3}{d}$  

$\frac{m^3}{d} = MGD \times \frac{1Mg}{d} \times\frac{g}{1M} \times \frac{.00378541 m^3}{g}$  

$\frac{MG}{d} --> \frac{m^3}{d} mult = .00378541 \times 1e6 = 3785.41$  

In [ ]:
well_df['flux_m3'] = [i for i in well_df.Avg_Mgd * (-3785.41)]
well_df['max_flux_m3'] = [i for i in well_df.M_Rate_MGD * (-3785.41)]

well_df

In [ ]:
well_df['flux_m3'].min()

In [ ]:
well_df['start_datetime'] = '2009-01-01'
well_df['end_datetime'] = '2016-01-01'

In [ ]:
well_df.drop(columns=['geometry'], inplace=True)

In [ ]:
well_df.to_csv('../processed_input/2009-2016_ave_pumping.csv')